In [1]:
path_xlsx_parte = r'C:\Documents\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\PARTE DIARIO TRICHOGRAMMA 2024_RES.xlsx'

In [2]:
import sys
sys.path.append('../_amigocloud')

In [3]:
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb

import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
#from shapely.geometry import Point, MultiPoint
import os
#from fastkml import kml

In [4]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [5]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [6]:
query = {'query': 'SELECT\
    g.id, p.idd, p.fecha_registro, p.piloto_1, p.piloto_2, p.hora_inicio, p.hora_fin, p.labor, p.temperatura, p.viento, p.humedad, p.estado_de_operacion, p.dron, p.obs FROM dataset_342996 p JOIN dataset_345601 g ON ST_Intersects(p.ubicaciones, g.geometry)'}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/33457/sql', query)
data = select['data']

In [7]:
df_lib = pd.DataFrame(data)

In [8]:
df_lib

,id,idd,fecha_registro,piloto_1,piloto_2,hora_inicio,hora_fin,labor,temperatura,viento,humedad,estado_de_operacion,dron,obs
0,220,57,2024-07-18 14:47:03+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 14:53:02+00:00,2024-07-18 15:53:02+00:00,LIBERACION,21,15,60,CONCLUIDO,MT 01,None
1,37,55,2024-07-18 11:49:44+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 18:19:44+00:00,2024-07-18 19:35:44+00:00,LIBERACION,22,20,65,CONCLUIDO,MT 01,None
2,2,54,2024-07-18 11:48:19+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-19 15:48:19+00:00,2024-07-19 17:18:19+00:00,LIBERACION,21,20,65,CONCLUIDO,MT 01,None
3,221,56,2024-07-18 12:43:14+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 12:51:14+00:00,2024-07-19 12:30:14+00:00,LIBERACION,19,2,60,CONCLUIDO,MT 01,None
4,3,54,2024-07-18 11:48:19+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-19 15:48:19+00:00,2024-07-19 17:18:19+00:00,LIBERACION,21,20,65,CONCLUIDO,MT 01,None
5,27,55,2024-07-18 11:49:44+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 18:19:44+00:00,2024-07-18 19:35:44+00:00,LIBERACION,22,20,65,CONCLUIDO,MT 01,None
6,33,55,2024-07-18 11:49:44+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 18:19:44+00:00,2024-07-18 19:35:44+00:00,LIBERACION,22,20,65,CONCLUIDO,MT 01,None
7,222,57,2024-07-18 14:47:03+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 14:53:02+00:00,2024-07-18 15:53:02+00:00,LIBERACION,21,15,60,CONCLUIDO,MT 01,None
8,4,54,2024-07-18 11:48:19+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-19 15:48:19+00:00,2024-07-19 17:18:19+00:00,LIBERACION,21,20,65,CONCLUIDO,MT 01,None
9,28,55,2024-07-18 11:49:44+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 18:19:44+00:00,2024-07-18 19:35:44+00:00,LIBERACION,22,20,65,CONCLUIDO,MT 01,None


In [9]:
lista_idd = list(set(df_lib['idd']))
lista_idd

[56, 57, 54, 55]

In [10]:
lista_id = list(set(df_lib['id']))
lista_id

[33, 2, 3, 4, 37, 28, 219, 29, 218, 27, 220, 221, 222, 223]

In [11]:
nulos = df_lib.isnull().sum()
nulos

id                      0
idd                     0
fecha_registro          0
piloto_1                0
piloto_2                0
hora_inicio             0
hora_fin                0
labor                   0
temperatura             0
viento                  0
humedad                 0
estado_de_operacion     0
dron                    0
obs                    14
dtype: int64

In [12]:
df_except_obs = df_lib.drop(columns=['obs'])
null_check = df_except_obs.isnull().any().any()

In [13]:
if null_check:
    df_lib = None
else:
    None

In [14]:
df_lib

,id,idd,fecha_registro,piloto_1,piloto_2,hora_inicio,hora_fin,labor,temperatura,viento,humedad,estado_de_operacion,dron,obs
0,220,57,2024-07-18 14:47:03+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 14:53:02+00:00,2024-07-18 15:53:02+00:00,LIBERACION,21,15,60,CONCLUIDO,MT 01,None
1,37,55,2024-07-18 11:49:44+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 18:19:44+00:00,2024-07-18 19:35:44+00:00,LIBERACION,22,20,65,CONCLUIDO,MT 01,None
2,2,54,2024-07-18 11:48:19+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-19 15:48:19+00:00,2024-07-19 17:18:19+00:00,LIBERACION,21,20,65,CONCLUIDO,MT 01,None
3,221,56,2024-07-18 12:43:14+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 12:51:14+00:00,2024-07-19 12:30:14+00:00,LIBERACION,19,2,60,CONCLUIDO,MT 01,None
4,3,54,2024-07-18 11:48:19+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-19 15:48:19+00:00,2024-07-19 17:18:19+00:00,LIBERACION,21,20,65,CONCLUIDO,MT 01,None
5,27,55,2024-07-18 11:49:44+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 18:19:44+00:00,2024-07-18 19:35:44+00:00,LIBERACION,22,20,65,CONCLUIDO,MT 01,None
6,33,55,2024-07-18 11:49:44+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 18:19:44+00:00,2024-07-18 19:35:44+00:00,LIBERACION,22,20,65,CONCLUIDO,MT 01,None
7,222,57,2024-07-18 14:47:03+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 14:53:02+00:00,2024-07-18 15:53:02+00:00,LIBERACION,21,15,60,CONCLUIDO,MT 01,None
8,4,54,2024-07-18 11:48:19+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-19 15:48:19+00:00,2024-07-19 17:18:19+00:00,LIBERACION,21,20,65,CONCLUIDO,MT 01,None
9,28,55,2024-07-18 11:49:44+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-18 18:19:44+00:00,2024-07-18 19:35:44+00:00,LIBERACION,22,20,65,CONCLUIDO,MT 01,None


In [15]:
df_parte = pd.read_excel(path_xlsx_parte)

In [16]:
df_parte

,N° LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,COD DRONE,PILOTO 1,PILOTO 2,OBSERVACIONES,CONFORMIDAD CANERO,FINANCIADO,ID,OS,IDD,REPORTE
0,641,2024-05-23,21.0,8,13171,SUAREZ JUSTINIANO DAVID,1958,NUEVA ESPERAZANZA - SUAREZ DAVID,L1,17.000000,...,DA 02,ALEJANDRO SANCHEZ,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
1,525,2024-06-27,26.0,66,1218,BALCAZAR ALCANTARA JOSE LOLY,510,LA BELLA & SANTA ROSA I & VILLA FLORES,L9.2,17.830000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,CREDITO 2024,NaN,NaN,NaN,SI
2,525,2024-06-27,26.0,66,1218,BALCAZAR ALCANTARA JOSE LOLY,510,LA BELLA & SANTA ROSA I & VILLA FLORES,L13,10.710000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,CREDITO 2024,NaN,NaN,NaN,SI
3,558,2024-06-27,26.0,124,1530,CITTCA,299,CITTCA,C10.3,2.490000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
4,558,2024-06-27,26.0,124,1530,CITTCA,299,CITTCA,C12.2,0.830000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,NaN,NaT,NaN,0,13067,SAUCEDO CERRUTI SONIA,111,NAICO--CERRUTI,L5,0.333723,...,NaN,NaN,NaN,NaN,NaN,SIEMBRA 2023,229.0,36.0,NaN,NaN
252,NaN,NaT,NaN,50,11860,ROCA SERRANO ARMANDO,114,SAUSALITO--ROCA,L7,73.439372,...,NaN,NaN,NaN,NaN,NaN,SOLICITUD,230.0,37.0,NaN,NaN
253,NaN,NaT,NaN,50,11860,ROCA SERRANO ARMANDO,114,SAUSALITO--ROCA,L5.7,3.558305,...,NaN,NaN,NaN,NaN,NaN,SOLICITUD,231.0,37.0,NaN,NaN
254,NaN,NaT,NaN,50,11860,ROCA SERRANO ARMANDO,114,SAUSALITO--ROCA,L5.9,3.481401,...,NaN,NaN,NaN,NaN,NaN,SOLICITUD,232.0,37.0,NaN,NaN


In [17]:
df_parte['FECHA'] = pd.to_datetime(df_parte['FECHA'], errors='coerce')
df_parte['HORA INICIO'] = pd.to_datetime(df_parte['HORA INICIO'], errors='coerce')
df_parte['HORA FINAL'] = pd.to_datetime(df_parte['HORA FINAL'], errors='coerce')

In [18]:
# Recorrer cada registro del DataFrame y actualizar los registros correspondientes en el archivo Excel
for idx, row in df_lib.iterrows():
    id_value = row['id']
    # Encontrar el índice correspondiente en el archivo Excel
    idx_parte = df_parte[df_parte['ID'] == id_value].index
    if not idx_parte.empty:
        idx_parte = idx_parte[0]
        # Convertir las fechas y eliminar la zona horaria
        fecha_registro = pd.to_datetime(row['fecha_registro'], errors='coerce').tz_localize(None)
        hora_inicio = pd.to_datetime(row['hora_inicio'], errors='coerce').tz_localize(None)
        hora_fin = pd.to_datetime(row['hora_fin'], errors='coerce').tz_localize(None)
        
        # Actualizar los valores en el archivo Excel
        df_parte.at[idx_parte, 'FECHA'] = fecha_registro
        df_parte.at[idx_parte, 'HORA INICIO'] = hora_inicio
        df_parte.at[idx_parte, 'HORA FINAL'] = hora_fin
        df_parte.at[idx_parte, 'TEMPERATURA'] = float(row['temperatura'])
        df_parte.at[idx_parte, 'VIENTO'] = float(row['viento'])
        df_parte.at[idx_parte, 'HUMEDAD'] = float(row['humedad'])
        df_parte.at[idx_parte, 'OPERACIÓN DETENIDA POR'] = str(row['estado_de_operacion'])
        df_parte.at[idx_parte, 'COD DRONE'] = str(row['dron'])
        df_parte.at[idx_parte, 'PILOTO 1'] = str(row['piloto_1'])
        df_parte.at[idx_parte, 'PILOTO 2'] = str(row['piloto_2'])
        df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])
        df_parte.at[idx_parte, 'IDD'] = float(row['idd'])
        df_parte.at[idx_parte, 'SEMANA'] = fecha_registro.isocalendar().week

C:\Users\bismarksr\AppData\Local\Temp\ipykernel_54508\2708270919.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])


In [19]:
output_path = path_xlsx_parte
df_parte.to_excel(output_path, index=False)

# HACER COPIA Y ELIMINAR PARTE DIARIO Y PLANIFICACION DE AMIGOCLOUD

In [20]:
lista_idd

[56, 57, 54, 55]

In [21]:
#copia y elimina los idd indicados del parte diario de amigocloud
for idd in lista_idd:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348546 (fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones) SELECT fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones FROM dataset_342996 WHERE idd = {idd}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_342996 WHERE idd = {idd}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso IDD:', idd)

se proceso IDD: 56
se proceso IDD: 57
se proceso IDD: 54
se proceso IDD: 55


In [22]:
#copia y elimina los id indicados de la planificacion semanal de amigocloud
for id in lista_id:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348547 (fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry) SELECT fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry FROM dataset_345601 WHERE id = {id}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_345601 WHERE id = {id}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso ID:', id)

se proceso ID: 33
se proceso ID: 2
se proceso ID: 3
se proceso ID: 4
se proceso ID: 37
se proceso ID: 28
se proceso ID: 219
se proceso ID: 29
se proceso ID: 218
se proceso ID: 27
se proceso ID: 220
se proceso ID: 221
se proceso ID: 222
se proceso ID: 223
